In [ ]:
!pip install emoji
!pip install contractions
!pip install spacy_langdetect
!pip install pycld2
!pip install simcse transformers==4.2.1 scipy==1.5.4 datasets==1.2.1 pandas==1.1.5 scikit-learn==0.24.0 prettytable==2.1.0 gradio torch setuptools==49.3.0

     |████████████████████████████████| 170 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=d62aaed24ab57a663c0f8030fd2cca9c99e63f127dca8c13b80beed9249379f2
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import json
import pandas as pd
from nltk import word_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import MWETokenizer
import zipfile
import re
from emoji import UNICODE_EMOJI
import os
import contractions
from time import time
from datetime import datetime
import csv
import torch
from simcse import SimCSE
import pandas as pd
import heapq
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = 'drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/'

In [ ]:
def preprocessing(main_zip_file, file, include_meta=True):
    data = None
    with zipfile.ZipFile(main_zip_file) as z:
        file = [x for x in z.namelist() if file in x]
        assert len(file)==1
        file = file[0]
        with z.open(file, 'r') as f:
            data = f.read()
    data = json.loads(data.decode('utf-8'))
    
    # rearrange - data into tweets
    data, user_metadata = restructure(data)
    data['orig_tweet'] = data['tweet']

    #ROUND 2: remove RT
    data = remove_rt(data)
    
    # replace urls to <url> - retain the info in another way?
    url_re = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%.\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}([-a-zA-Z0-9()@:%\+.~#?&//=]*)'
    replace_in_tweet(data, url_re, ' <url> ')
    
    # replace @userhandle with <user>
    # replace_in_tweet(data, '@\w+', ' <user> ')
    
    # replace #hashtag with <hashtag>
    # replace_in_tweet(data, '#\w+', ' <hashtag> ')
    
    # replace numbers with <number>
    # replace_in_tweet(data, '\d+', ' <number> ')
    
    # replace emojis with tags
    data['tweet'] = data['tweet'].apply(space_out_emojis)
    
    #removing extra spaces
    data['tweet'] = data['tweet'].str.replace('\s+', ' ')
    data['tweet'] = data['tweet'].apply(lambda x: x.strip())

    #handle contractions
    # print('Starting contractions')
    def fn(x):
        res = x
        try:
            res = contractions.fix(x)
        except Exception as e:
            pass #seems to be from other languages - handle if not filtering?
        return res
    data['tweet'] = data['tweet'].apply(lambda x: fn(x))
    # print('Handling contractions')
    
    # tokenize words
    # mwe_tokens = MWETokenizer([('<','user','>'),('<','hashtag','>'), ('<','url','>'), ('<','number','>')])
    # data['tweet'] = data['tweet'].apply(lambda x: mwe_tokens.tokenize(word_tokenize(x)))
    
    # replace all uppercase - token+<allcaps> - another way to handle case?
    # data['tweet'] = data['tweet'].apply(handle_allcaps)
    
    # replace lowercase
    # data['tweet'] = data['tweet'].apply(lambda x: x.lower())

    data['tweet'] = data['tweet'].apply(handle_extended)

    #ROUND 3: metadata
    if include_meta:
        add_metadata(data, user_metadata)
    
    # replace same letter repeated 3+ times - needed?
    # add info to word vectors - variable - future work
    return data, user_metadata

In [ ]:
# def handle_allcaps(tweet):
#     tokens = word_tokenize(tweet)
#     for token in tokens:
#         if token.isupper():
#             tweet = tweet.replace(token, token.lower()+'_allcaps_')
#     return tweet

In [ ]:
def handle_extended(tweet):
    tokens = word_tokenize(tweet)
    for token in tokens:
        res=re.sub(r'([a-zA-Z])\1{2,}', r'\1', token)
        if res!=token:
            tweet = tweet.replace(token, token+"<ext>")
    return tweet
    # re.sub(r'([a-zA-Z])\1{2,}', r'\1', 'whaaaaaaaat')
    #extended tokens

print(handle_extended("whaaaaaattttt are you saying. are you crazzzzyyyyyyyyy!!! Loooooookkkk!!!!"))

whaaaaaattttt<ext> are you saying. are you crazzzzyyyyyyyyy<ext>!!! Loooooookkkk<ext>!!!!


In [ ]:
def space_out_emojis(tweet):
    all_emojis = UNICODE_EMOJI['en'].keys()
    tweet_emojis = {x for x in all_emojis if x in tweet}
    for emoji in tweet_emojis:
        tweet = re.sub(re.escape(emoji), ' '+emoji+' ', tweet)
    return tweet

In [ ]:
def restructure(data):
    tweet_data = []
    user_data = []
    for instance in data:
        if type(instance['tweet']) != type(None):
            for tweet in instance['tweet']:
                tweet_data.append( [instance['ID'], tweet, instance['label']] )
        user_meta = eval(str(instance))
        del user_meta['tweet']
        user_data.append(user_meta)
    return pd.DataFrame(tweet_data, columns=['id', 'tweet', 'label']), user_data

In [ ]:
def replace_in_tweet(data, re_to_replace, replace_with):
    data['tweet'] = data['tweet'].apply(lambda x: re.sub(re_to_replace, replace_with, x))

In [ ]:
def write_triplets(data, filename):
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))
    bots = data[data['label']=='1'].sample(30000)['tweet']
    humans = data[data['label']=='0'].sample(30000)['tweet']
    out1 = pd.DataFrame(zip(bots[:10000], bots[10000:20000], humans[20000:]), columns=['sent0', 'sent1', 'hard_neg'])
    out2 = pd.DataFrame(zip(humans[:10000], humans[10000:20000], bots[20000:]), columns=['sent0', 'sent1', 'hard_neg'])
    out = pd.concat([out1, out2])
    out.to_csv(filename, index=False)

In [ ]:
# def write_triplets_with_similarity(data, filename):
#     data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))
#     bots_tweets = data[data['label']=='1']['tweet']
#     humans_tweets = data[data['label']=='0']['tweet']

#     #For each tweet, find top 2 similar positive and top 2 similar negative
#     out1 = pd.DataFrame(zip(bots[:10000], bots[10000:20000], humans[20000:]), columns=['sent0', 'sent1', 'hard_neg'])
#     out2 = pd.DataFrame(zip(humans[:10000], humans[10000:20000], bots[20000:]), columns=['sent0', 'sent1', 'hard_neg'])
#     out = pd.concat([out1, out2])
#     out.to_csv(filename, index=False)

In [ ]:
def remove_rt(data):
    columns=['id', 'tweet', 'label', 'orig_tweet']
    data['is_retweet'] = data['orig_tweet'].apply(lambda x: x[:3] == 'RT ')
    bots = data[data['label']=='1']
    humans = data[data['label']=='0']
    print('Ratio of RT tweets by bots:', len(bots[bots['is_retweet']]), '/', len(bots), '(',len(bots[bots['is_retweet']])*100/len(bots), '%)' )
    print('Ratio of RT tweets by humans:', len(humans[humans['is_retweet']]), '/', len(humans), '(',len(humans[humans['is_retweet']])*100/len(humans), '%)'  )
    data = data[~(data['is_retweet'])].copy()
    data.drop(columns=['is_retweet'], inplace=True)
    return data

In [ ]:
def add_metadata(data, user_metadata):

    metadata = {x['ID']:x['profile'] for x in user_metadata}
    #variable length of user_name and text desc, profile image url
    def concat_meta(prof):
        params= ["screen_name", "protected", "verified", "default_profile", "default_profile_image",
                  "followers_count", "friends_count", "listed_count", "favourites_count", "statuses_count"]
        meta_text = [ prof[x].strip() for x in params ]
        meta_text[1] = 'protected' if meta_text[1]=='True' else 'unprotected'
        meta_text[2] = 'verified' if meta_text[2]=='True' else 'unverified'
        meta_text[3] = 'defprof' if meta_text[3]=='True' else 'nondefprof'
        meta_text[4] = 'defimg' if meta_text[4]=='True' else 'nondefimg'
        return ' '.join( meta_text )

    data['tweet'] = data.apply(lambda x: concat_meta(metadata[x['id']]) + " " + x['tweet'], axis=1)

In [ ]:
# for dataset in ('train', 'dev', 'test'):
#     data, user_metadata = preprocessing(folder_path+'Twibot-20.zip', dataset+'.json')
#     write_triplets(data, folder_path+dataset+'_triplets.csv')

In [ ]:
def generate_preprocessed_data(dataset):
    start = time()
    data, user_metadata = preprocessing(folder_path+'Twibot-20.zip', dataset+'.json')
    # data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))
    print('Remaining bot tweets:', len(data[data['label']=='1']))
    print('Remaining human tweets:', len(data[data['label']=='0']))
    print('Time taken:', (time() - start))
    return data

In [ ]:
# dataset = 'test'
# data, user_metadata = preprocessing(folder_path+'Twibot-20.zip', dataset+'.json')
# # data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))
# print('Remaining bot tweets:', len(data[data['label']=='1']))
# print('Remaining human tweets:', len(data[data['label']=='0']))
# data

In [ ]:
# dataset = 'dev'
# data, user_metadata = preprocessing(folder_path+'Twibot-20.zip', dataset+'.json')
# # data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))
# print('Remaining bot tweets:', len(data[data['label']=='1']))
# print('Remaining human tweets:', len(data[data['label']=='0']))

Ratio of RT tweets by bots: 78591 / 212950 ( 36.905846442826956 %)
Ratio of RT tweets by humans: 48586 / 188575 ( 25.764815060320828 %)
Remaining bot tweets: 134359
Remaining human tweets: 139989


In [ ]:
# joined_data = pd.concat([spacy_ref, data], axis=1)
def join_spacy_data(data, dataset):
    spacy_ref = pd.read_csv(os.path.join(folder_path, 'english_labelled_ref_data', dataset+'_tweets_spacy_labelled.csv'))
    assert len(data)==len(spacy_ref), "Row mismatched with ref data"
    # assert len
    cols = spacy_ref.columns
    spacy_ref.rename(columns={x:x+'_spc' for x in cols}, inplace=True)
    print(len(spacy_ref))
    joined = pd.merge(spacy_ref, data, left_index=True, right_index=True)
    print(len(joined))
    joined = joined[['id', 'tweet', 'label', 'orig_tweet', 'lang_spc', 'lang_code_spc']]
    return joined
# joined[joined['orig_tweet_x']==joined['orig_tweet_y']].count()
# data.reset_index(inplace=True, drop=True)

In [ ]:
for dataset in ['train', 'test', 'dev']:
    data = generate_preprocessed_data(dataset)
    data.reset_index(inplace=True)
    joined_data = join_spacy_data(data, dataset)
    joined_data.to_csv(os.path.join(folder_path, 'data_round_3_corrected', dataset+'_tweets_spacy_round3.csv'), index=False)

Ratio of RT tweets by bots: 289171 / 756619 ( 38.218839336574945 %)
Ratio of RT tweets by humans: 163623 / 641791 ( 25.494748290331277 %)
Remaining bot tweets: 467448
Remaining human tweets: 478168
Time taken: 548.1311504840851
945616
945616
Ratio of RT tweets by bots: 39970 / 102927 ( 38.83334790676888 %)
Ratio of RT tweets by humans: 25391 / 96926 ( 26.196273445721477 %)
Remaining bot tweets: 62957
Remaining human tweets: 71535
Time taken: 79.39990496635437
134492
134492
Ratio of RT tweets by bots: 78591 / 212950 ( 36.905846442826956 %)
Ratio of RT tweets by humans: 48586 / 188575 ( 25.764815060320828 %)
Remaining bot tweets: 134359
Remaining human tweets: 139989
Time taken: 159.53771948814392
274348
274348


In [ ]:
spacy_ref = pd.read_csv(os.path.join(folder_path, 'english_labelled_ref_data', 'train_tweets_spacy_labelled.csv'))
len(spacy_ref[spacy_ref['lang_code']=='en']), len(spacy_ref)

(623464, 945616)

In [ ]:
# dataset = 'test'
# spacy_data = pd.read_csv(os.path.join(folder_path, 'english_labelled_ref_data', dataset+'_tweets_spacy_labelled.csv'))
# spacy_data['lang'] = spacy_data['lang'].apply(lambda x: eval(x))
# spacy_data['lang_code'] = spacy_data['lang'].apply(lambda x: x['language'] if x['score']>=0.99 else 'unsure') # run for dev, test

In [ ]:
# spacy_data[spacy_data['lang_code']=='so'] #can cross check with pycld2 to classify better
# spacy_data['lang_code'].unique()
spacy_data.to_csv(os.path.join(folder_path, 'english_labelled_ref_data', dataset+'_tweets_spacy_labelled_new.csv'), index=False)

In [ ]:
from spacy_langdetect import LanguageDetector
import spacy
def find_spacy_labels(data):
    lang_detect = spacy.load('en') #defaults to english
    lang_detect.add_pipe(LanguageDetector(), name='language_detector', last=True)

    def detect_language(text, lang_detect):
        doc = lang_detect(text)
        return doc._.language
    data['lang'] = data['tweet'].apply(lambda x: detect_language(x, lang_detect))
    data['lang_code'] = data['lang'].apply(lambda x: x['language'] if x['score']>=0.99 else 'unsure')

In [ ]:
joined_data

,id,tweet,label,orig_tweet,lang_spc,lang_code_spc
1,17461978,SHAQ False True False False 15349596 692 45568...,0,Who has time for receipts? Not me. @epson rece...,"{'language': 'en', 'score': 0.9999971783640074}",en
2,17461978,SHAQ False True False False 15349596 692 45568...,0,Steady wants to encourage you to invest in you...,"{'language': 'en', 'score': 0.9999972659917247}",en
3,17461978,SHAQ False True False False 15349596 692 45568...,0,"Good one, @rishid. But let’s see if y'all can ...","{'language': 'sq', 'score': 0.7142845190395024}",unsure
4,17461978,SHAQ False True False False 15349596 692 45568...,0,#lsunationalchamps\n,"{'language': 'en', 'score': 0.9999970673587957}",en
5,17461978,SHAQ False True False False 15349596 692 45568...,0,I stand with the student athletes! #WeWantToPl...,"{'language': 'en', 'score': 0.9999974093430682}",en
...,...,...,...,...,...,...
945608,926345425,Tamare_xo False False False False 824 456 1 16...,0,Getting ready and realize I can’t find my lash...,"{'language': 'en', 'score': 0.9999956733790624}",en
945609,926345425,Tamare_xo False False False False 824 456 1 16...,0,Parents can be so judgemental sometimes\n,"{'language': 'en', 'score': 0.9999975935611402}",en
945610,926345425,Tamare_xo False False False False 824 456 1 16...,0,@djshella876_ Me woulda more likely fi tell yo...,"{'language': 'en', 'score': 0.9999967992298511}",en
945611,926345425,Tamare_xo False False False False 824 456 1 16...,0,🤔 https://t.co/Rn2j0xoM6q\n,"{'language': 'en', 'score': 0.9999977318834589}",en


In [ ]:
# joined['ext'] = joined['tweet'].apply(lambda x: re.search( , x))
# re.sub(r'([a-zA-Z])\1{2,}', r'\1', 'whaaaaaaaat')

In [ ]:
#also try spacy, pycld2: https://towardsdatascience.com/4-python-libraries-to-detect-english-and-non-english-language-c82ad3efd430
# from spacy_langdetect import LanguageDetector
# import spacy
# lang_detect = spacy.load('en') #defaults to english
# lang_detect.add_pipe(LanguageDetector(), name='language_detector', last=True)

# def detect_language(text, lang_detect):
#     doc = lang_detect(text)
#     return doc._.language

# TRY IF WE NEED MULTIPLE LANGUAGES:
# start = time()
# import pycld2 as cld2
# text_content = "Er lebt mit seinen Eltern und seiner Schwester in Berlin. Welcome, to this world of Data Scientist. Today is a lovely day."
# text_content = """ A accès aux chiens et aux frontaux qui lui ont été il peut consulter et modifier ses collections et exporter Cet article concerne le pays européen aujourd’hui appelé République française. 
# Pour d’autres usages du nom France, Pour une aide rapide et effective, veuiller trouver votre aide dans le menu ci-dessus. 
# Welcome, to this world of Data Scientist. Today is a lovely day."""
# def detect_language(text):
#     detected_language = ((0, 0, 'f', 'failed_char'),)
#     try:
#         # check_for = {'', ''} #\x92, \x82
#         # for ch in check_for:
#         #     if ch in text:
#         #         text = re.sub(ch, '', text)
#         _, _, _, detected_language = cld2.detect(text,  returnVectors=True)
#     except Exception as e:
#         print(text, e, '\n')
#     if not detected_language:
#         return 'en'
#     return max(detected_language, key=lambda x:x[1])[-1]

# data['lang'] = data['tweet'].apply(lambda x: detect_language(x))
# print(time() - start)


In [ ]:
# start = time()
# dataset = 'dev'
# data, user_metadata = preprocessing(folder_path+'Twibot-20.zip', dataset+'.json', include_meta=False)
# data['lang'] = data['tweet'].apply(lambda x: detect_language(x, lang_detect))
# data.to_csv(folder_path+'dev_tweets_spacy_labelled.csv', index=False)
# print(time() - start)

Ratio of RT tweets by bots: 78591 / 212950 ( 36.905846442826956 %)
Ratio of RT tweets by humans: 48586 / 188575 ( 25.764815060320828 %)
5865.719421386719


In [ ]:
# start = time()
# dataset = 'test'
# data, user_metadata = preprocessing(folder_path+'Twibot-20.zip', dataset+'.json', include_meta=False)
# data['lang'] = data['tweet'].apply(lambda x: detect_language(x, lang_detect))
# data.to_csv(folder_path+'test_tweets_spacy_labelled.csv', index=False)
# print(time() - start)

Ratio of RT tweets by bots: 39970 / 102927 ( 38.83334790676888 %)
Ratio of RT tweets by humans: 25391 / 96926 ( 26.196273445721477 %)
2902.9452855587006


In [ ]:
#*********************** ROUND 2 **************************

In [ ]:
torch.cuda.is_available()
dtype = torch.cuda.FloatTensor

In [ ]:
data_folder = 'drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/'

In [ ]:
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")
## is supervised and case insensitive

In [ ]:
data = pd.read_csv(data_folder+'train_tweets_spacy_labelled.csv')

In [ ]:
data = data[data['lang_code']=='en'].reset_index(drop=True)
data = data.drop(columns=['orig_tweet', 'lang', 'lang_code'])

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
print(dir(model))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'build_index', 'device', 'encode', 'index', 'is_faiss_index', 'model', 'num_cells', 'num_cells_in_search', 'pooler', 'search', 'similarity', 'tokenizer']


In [ ]:
tweet_sample = data.sample(1000)['tweet'].to_list()

In [ ]:
start = time()
sim_mat = model.similarity(tweet_sample, tweet_sample)
print( time()-start )

## Test similarity matrix for 1000 tweets

In [ ]:
# Get top 3 similar - the most similar entity will be the entity itself. We consider the 2nd and 3rd indices as the 1st two most similar sentences.

import numpy as np
import heapq

print(tweet_sample[999])
# print(sim_mat[999])
print(np.argmax(sim_mat[999]))
print(tweet_sample[np.argmax(sim_mat[999])])
top3 = heapq.nlargest(3, sim_mat[999])
print(top3)
top3_indices = heapq.nlargest(3, range(len(sim_mat[999])), key=sim_mat[999].__getitem__)
print(top3_indices)

for idx in top3_indices:
  print(tweet_sample[idx])


wipe your tears <_url_>
999
wipe your tears <_url_>
[1.0000002, 0.55124474, 0.5318104]
[999, 360, 564]
wipe your tears <_url_>
love you al , thank you 💓 💓 💓 <_url_>
<_user_> so sorry for your loss . may she rest in peace and may you all find peace and cherish the memories .


## Build a similarity matrix for all tweets in the dataset (post processing the dataset)
Takes about 1 hour to build the similarity matrix for 600k x 600k (all tweets x all tweets ) and
Takes about 15 minutes to build the similarity matrix for 1 x 325k (one tweet row x all bot tweets)

In [ ]:
# Not enough memory to pre-compile similarity matrices - even for just all_tweets x bot_tweets

sim_mat_time_start = time()
# # data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))

all_tweets = data['tweet'].to_list()
print(len(all_tweets))
bot_tweets = data[data['label']==1]['tweet'].to_list()
human_tweets = data[data['label']==0]['tweet'].to_list()
print("bot_tweets", len(bot_tweets))
print("human_tweets", len(human_tweets))
print(len(bot_tweets) + len(human_tweets))

# Build a similarity matrix for all tweets with bot tweets
# sim_mat_bot = model.similarity(all_tweets, bot_tweets)
sim_mat_bot = model.similarity(all_tweets[0], bot_tweets)
print(len(sim_mat_bot))

# # Build a similarity matrix for all tweets with human tweets



print("\nTime taken to build similarity matrix for 1 row", time() - sim_mat_time_start)

11/24/2021 20:31:13 - INFO - numexpr.utils -   NumExpr defaulting to 4 threads.


623464
bot_tweets 325841
human_tweets 297623
623464


100%|██████████| 5092/5092 [14:42<00:00,  5.77it/s]


325841

Time taken to build similarity matrix for 1 row 894.5402007102966


## Build similarity matrices for 50k tweets. One sim matrix for tweets x bot_tweets, another for tweets x human_tweets

In [ ]:
# Include labels while writing to the triplets CSV!

sim_mat_time_start = time()
# data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x))

# As of now, we sample 50000 random rows with seed = 1. If we wanna progressively build the entire dataset, we can take the first 50000 rows, and so on.
data_subset = data[['tweet','label']].sample(n=50000, random_state=1)
tweet_subset = data_subset['tweet'].to_list()
label_subset = data_subset['label'].to_list()
print(data_subset[:5])
print(data_subset[-5:])
bot_tweets = data_subset[data_subset['label']==1]['tweet'].to_list()
bot_tweet_labels = data_subset[data_subset['label']==1]['label'].to_list()
human_tweets = data_subset[data_subset['label']==0]['tweet'].to_list()
human_tweet_labels = data_subset[data_subset['label']==0]['label'].to_list()
print("bot_tweets", len(bot_tweets))
print("human_tweets", len(human_tweets))
print("total tweets", len(bot_tweets) + len(human_tweets))

11/22/2021 23:55:20 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.


                                                    tweet  label
446581  brazilian govt . pretending to protect highly ...      1
208653  <_user_> she is the one inviting folks over 😂 ...      0
124024  back in lithuania at the end of this month <_u...      0
26152   chance the rapper is set to make a movie music...      1
450671  <_user_> you are such a savior of the common m...      1
                                                    tweet  label
513085  <_user_> russians are so sneaky ... they trick...      1
210624  <_user_> <_user_> his team does not want him t...      1
399449  <_user_> i was completely wrong about him but ...      1
227756  <_user_> congrats again . not sure if you got ...      1
144317  <_url_> check this short out . my son wrote an...      0
bot_tweets 26278
human_tweets 23722
total tweets 50000


In [ ]:
# Build a similarity matrix for all tweets with bot tweets

# sim_mat_bot = model.similarity(tweet_subset, bot_tweets)
# print(len(sim_mat_bot))

# np.save(data_folder+'similarity_matrix_bot_50k.npy', sim_mat_bot)

In [ ]:
# # Build a similarity matrix for all tweets with human tweets
# sim_mat_human = model.similarity(tweet_subset, human_tweets)
# print(len(sim_mat_human))
# np.save(data_folder+'similarity_matrix_human_50k.npy', sim_mat_human)

In [ ]:
sim_mat_bot = np.load(data_folder+'similarity_matrix_bot_50k.npy')
sim_mat_human = np.load(data_folder+'similarity_matrix_human_50k.npy')

## Build triplet CSV (dataset size = 50k right now) 
# Prev exec : 6622.105570077896 seconds

In [ ]:

triplet_csv_build_start = time()

out = pd.DataFrame(columns=['sent0', 'sent1', 'hard_neg', 'sent0_label', 'sent1_label', 'hard_neg_label'])
print("tweets in tweet_subset", len(tweet_subset))
for idx in range(len(data_subset)):
  if idx % 100 == 0:
    print("{}/{}".format(idx, len(data_subset)))
  # find top n similar bot tweets
  top3_bot_indices = heapq.nlargest(3, range(len(sim_mat_bot[idx])), key=sim_mat_bot[idx].__getitem__)
  # # find top n similar human tweets
  top3_human_indices = heapq.nlargest(3, range(len(sim_mat_human[idx])), key=sim_mat_human[idx].__getitem__)

  # combine all the permutations of the most similar bot and human tweets
  # ignore the first most similar bot tweet if current tweet is a bot tweet
  # else ignore the first most similar human tweet
  if label_subset[idx]==1:
    for bot_idx in top3_bot_indices[:2]:
      for human_idx in top3_human_indices[:2]:
        # print("<bot>",tweet_subset[idx], tweet_subset[bot_idx], tweet_subset[human_idx])
        out = out.append({'sent0': tweet_subset[idx], 'sent1': bot_tweets[bot_idx], 'hard_neg': human_tweets[human_idx],\
                          'sent0_label': label_subset[idx], 'sent1_label': bot_tweet_labels[bot_idx], 'hard_neg_label': human_tweet_labels[human_idx]}, ignore_index=True)
        # print(out_line)
        # out.append(out_line)

  elif label_subset[idx]==0:
    for bot_idx in top3_bot_indices[:2]:
      for human_idx in top3_human_indices[1:]:
        # print("<human>",tweet_subset[idx], tweet_subset[bot_idx], tweet_subset[human_idx])
        out = out.append({'sent0': tweet_subset[idx], 'sent1': human_tweets[human_idx], 'hard_neg': bot_tweets[bot_idx],\
                          'sent0_label': label_subset[idx], 'sent1_label': human_tweet_labels[human_idx], 'hard_neg_label': bot_tweet_labels[bot_idx]}, ignore_index=True)
        # print(out_line)
        # out.append(out_line)
  else:
    print("something went wrong in the label comparison")

print(out)

out.to_csv(data_folder+"train_triplets_with_similarity.csv", index=False)

print('Time taken to build CSV:', (time() - triplet_csv_build_start))

tweets in tweet_subset 50000
                                                    sent0  ... hard_neg_label
0       brazilian govt . pretending to protect highly ...  ...              0
1       brazilian govt . pretending to protect highly ...  ...              0
2       brazilian govt . pretending to protect highly ...  ...              0
3       brazilian govt . pretending to protect highly ...  ...              0
4       <_user_> she is the one inviting folks over 😂 ...  ...              1
...                                                   ...  ...            ...
199995  <_user_> congrats again . not sure if you got ...  ...              0
199996  <_url_> check this short out . my son wrote an...  ...              1
199997  <_url_> check this short out . my son wrote an...  ...              1
199998  <_url_> check this short out . my son wrote an...  ...              1
199999  <_url_> check this short out . my son wrote an...  ...              1

[200000 rows x 6 columns]
Time tak